In [84]:
using Pipe
using DataFrames
# using StatsPlots
using CSV
# using RollingFunctions
using Query
# using GLMakie

using JSON
using DataFrames
using Dates
using Colors
# using ColorSchemes
using Indicators

import Colors
using Plots
using Interact
using JuliaDB
using Interact
# import Pkg; Pkg.add("ShiftedArrays")
# using ShiftedArrays
include("utils.jl")

window_slider (generic function with 1 method)

window_slider (generic function with 1 method)

In [3]:
json_data_folder = "../user_data/data/binance"
json_filename = "BTC_USDT-1m.json"
json_filepath = "$json_data_folder/$json_filename"
f = open(json_filepath, "r")
json_data = JSON.parse(f)
close(f)

In [4]:
dfa = DataFrame(unix_time = Int64[], open=Float64[], high=Float64[], low=Float64[], close=Float64[], volume=Float64[])
for r in json_data push!(dfa, r) end
dfa.unix_time .= dfa.unix_time ./1000
# dfa.time .= unix2datetime.(dfa.unix_time)

insertcols!(dfa, :time => unix2datetime.(dfa.unix_time))
nrow(dfa)

129177

In [5]:
start_date = DateTime(2021, 5, 1)
end_date = DateTime(2021, 5, 27)
vdf = dfa[(dfa.time .>= start_date) .& (dfa.time .<= end_date) , :]
ut, tt, hh, ll, cc = vdf.unix_time, vdf.time, vdf.high, vdf.low, vdf.close
# h14max = [fill(vdf[1,:high],8-1)...,rollmax(vdf.high, 8)...]
#  length(h14max), length(vdf.close)
# hlc = [hh ll cc]
length(tt)

37441

In [53]:
ar = cc
# ema = Indicators.ema
# ema26 = ema(ar,n=26)
# ema50 = ema(ar,n=330)
ema100 = ema(ar,n=100)
# ema200 = ema(ar,n=200)
# sma = Indicators.sma
# sma26 = sma(ar,n=26)
# sma50 = sma(ar,n=50)
sma100 = sma(ar,n=100)
# sma200 = sma(ar,n=200)
# sma400 = sma(ar,n=400)
# sma800 = sma(ar,n=800)
1

1

In [119]:
function calc_wavelet(ar, m)
    wt = wavelet(WT.haar)
    xt = dwt(ar, wt)
    threshold!(xt, BiggestTH(), m)
    fxt = idwt(xt, wt)
end

function get_wavelet(ar, (m, perc, shift))
    fxt = calc_wavelet(ar, m)
    
    shifted_wave1 = (1 + perc/100) * 
        circshift(fxt , Int(trunc(m * shift)))

end

get_wavelet (generic function with 1 method)

In [96]:

function wavelet_sliders()
    
    m_slider = slider(1:200; label="m", default=50)
    perc_slider = slider(-5.0:0.05:5.0; label="%", default=0)
    shift_slider = slider(-1:0.05:1.0; label="shift", default=0)
    
    output = Interact.@map (&m_slider , &perc_slider, &shift_slider)
#     output = Interact.@map circshift(get_wavelet(ar, m)*(1+ perc/100), Int(trunc(m*shift)) )
    
    wdg = Widget(["m_slider" => m_slider, "perc_slider" => perc_slider, "shift_slider" => shift_slider], output = output)
    @layout! wdg hbox(:m_slider, :perc_slider, :shift_slider)
end

wavelet_sliders (generic function with 2 methods)

In [108]:
wave1[][100]

48910.800000000076

In [126]:
using Wavelets
using LinearAlgebra

arlen = length(ar)
wnd = window_slider(arlen)

rng = Observable{AbstractRange{Int64}}(1:1)
map!(wnd -> wnd[1]*wnd[2]:(wnd[1]+1)*wnd[2]-1, rng, wnd)

par = Interact.@map ar[&rng]

wave_slds_1 = wavelet_sliders()
wave_1 = Interact.@map get_wavelet(&par, &wave_slds_1) 


wave_slds_2 = wavelet_sliders()
wave_2 = Interact.@map get_wavelet(&par, &wave_slds_2) 

# wave1 = Interact.@map get_wavelet(&par, &wave_slds_1[:m_slider]) 

# shifted_wave1 = Interact.@map (1 + &wave_slds_1[:perc_slider]/100) * 
#     circshift(&wave1 , Int(trunc(&wave_slds_1[:m_slider] * &wave_slds_1[:shift_slider])))



plot_box = Observable{Any}(dom"div"())

function draw_it_all(zar, wave_1, wave_2)    
    plt = plot(zar, size=(1200,600), legend=nothing)
    plt1 = plot!(wave_1, lw=1.5)
    plt1 = plot!(wave_2, lw=1.5)
    dom"div"(plt1)
end

# function draw_it_all(ar, (m, perc, shift))
    
#     wave_ar = get_wavelet(ar, m)
#     plt = plot(ar, size=(1200,600), legend=nothing)
#     plt1 = plot!(circshift(wave_ar*(1+ perc/100), Int(trunc(m*shift)) ), color=:red)
#     dom"div"(plt1)
# end

map!(draw_it_all, plot_box, par, wave_1, wave_2)
# vline!([5,10])
# wndplot
ui = dom"div"(wnd, wave_slds_1, wave_slds_2, plot_box)

(div
  Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["2^n els/wnd"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol, Any}(:max => 15, :default => 7.5, :attributes => Dict{Any, Any}(:type => "range", Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}", "orient" => "horizontal"), :min => 1, :step => 1, :className => "slider slider is-fullwidth", :style => Dict{Any, Any}()))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{Observables.AbstractObservable, Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing), "index" => (Observable{Int64} with 2 listeners. Value:
8, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/Users/johnsmith/.julia/packages/Knockout/IP1uR/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/johnsmith/.julia/packages/Knockout/IP1uR/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/johnsmith/.julia/packages/InteractBase/Zfu5P/src/../assets/all.js"), Asset("css", nothing, "/Users/johnsmith/.julia/packages/InteractBase/Zfu5P/src/../assets/style.css"), Asset("css", nothing, "/Users/johnsmith/.julia/packages/Interact/SbgIk/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Condition(Base.InvasiveLinkedList{Task}(Task (runnable) @0x000000014aa84bc0, Task (runnable) @0x000000014aa84bc0), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"1\",\"2\",\"3\",\"4\",\"5\",\"6\",\"7\",\"8\",\"9\",\"10\",\"11\",\"12\",\"13\",\"14\",\"15\"],\"changes\":WebIO.getval({\"name\":\"changes\",\"scope\":\"10245878831783682722\",\"id\":\"4279644296075909553\",\"type\":\"o

In [45]:

# mp = manipulate for 
#             m in slider(1:30; label="m", default=7)
#             ixs in slider(1:xmax; label="ixs", default=64)

#     n = 2^J
#     vbox(wnd, plt[])
#     plt[]
#     ixs = xmax
#     ixs = _ixs #Int(trunc(min(m * 2 * _ixs, xmax)))
#     wt = wavelet(WT.haar)
    # x = sin.(4*range(0, stop=2*pi-eps(), length=1024))
    # x = copy(cc)/(maximum(cc) + 1)
#     tree = bestbasistree(y[1:ixs], wt)
#     ytb = wpt(y[1:ixs], wt, tree)
#     xt = dwt(ar[rng[]], wt)
    # get biggest m-term approximations
#     m = 30
    
#     threshold!(ytb, BiggestTH(), m)
#     threshold!(xt, BiggestTH(), m)
#     plot!(xt)
    # compare sparse approximations in ell_2 norm
    # norm(x - iwpt(xtb, wt, tree), 2) # best basis wpt
    # norm(x - idwt(xt, wt), 2)        # regular dwt
#     xidwt = idwt(xt, wt)
#     , legend = :outertopright
#     plot(ix, y * ymax, lw=1, size=(1200,400), xlims=(1,xmax), color=:blue)
#     plot!(ix[1:ixs],yidwt[1:ixs] * ymax)
#     plot!(ix[1:ixs],yidwt[1:ixs] * ymax,fillrange=yidwt[1:ixs] * ymax * 0.99, fillalpha=0.2)
#     plot(x)
#     dnx = denoise(x,TI=true)
#     plot!(dnx)
    
#     wplotim(x::AbstractArray, L::Integer, wt::Union{DiscreteWavelet,Nothing}=nothing;
#                 wabs::Bool=true, power::Real=0.7, pnorm::Real=1)

#     A = wplotim(x)
#     Plots.scatter(d,l)
#     plot(x)
#     plt
    
# end 
# vbox(wnd,mp)
# mp

In [70]:
 [ (i,j, j/60, j/60/24) for (i,j) in [(i, 2^i) for i in 1:15]]

15-element Vector{Tuple{Int64, Int64, Float64, Float64}}:
 (1, 2, 0.03333333333333333, 0.001388888888888889)
 (2, 4, 0.06666666666666667, 0.002777777777777778)
 (3, 8, 0.13333333333333333, 0.005555555555555556)
 (4, 16, 0.26666666666666666, 0.011111111111111112)
 (5, 32, 0.5333333333333333, 0.022222222222222223)
 (6, 64, 1.0666666666666667, 0.044444444444444446)
 (7, 128, 2.1333333333333333, 0.08888888888888889)
 (8, 256, 4.266666666666667, 0.17777777777777778)
 (9, 512, 8.533333333333333, 0.35555555555555557)
 (10, 1024, 17.066666666666666, 0.7111111111111111)
 (11, 2048, 34.13333333333333, 1.4222222222222223)
 (12, 4096, 68.26666666666667, 2.8444444444444446)
 (13, 8192, 136.53333333333333, 5.688888888888889)
 (14, 16384, 273.06666666666666, 11.377777777777778)
 (15, 32768, 546.1333333333333, 22.755555555555556)

In [ ]:
n = length(ar)
wnd = window_slider(20000)
rng = Observable{AbstractRange{Int64}}(1:1)
map!(wnd -> wnd[1]*wnd[2]:(wnd[1]+1)*wnd[2]-1, rng, wnd)
plt = Observables.@map plot(ar[&rng])
A = Observables.@map length(&rng) > 2 ? heatmap(wplotim(ar[&rng])) : nothing
hbox(plt,A)

In [ ]:
length(1:3)

In [ ]:
length(ar)